In [292]:
import pandas as pd
import numpy as np

data=pd.read_csv(r'C:\Users\Reyan\Desktop\Proj\Apple\Data.csv')
data=data.sample(frac=1).reset_index(drop=True)

train_input_data=data.drop(['O'],axis=1)
train_input=np.matrix(train_input_data.values)
train_output_data=data.drop(['A','B','C','D'],axis=1)
train_output=np.matrix(train_output_data.values)
train_input[0].shape

(1, 4)

In [293]:
#set nn structure [input nodes,x hiddenlayers with x nodes....,output nodes, o for offset]
#function to generate random chromosomes
def chromosome_generetor(structure):
    x=[]
    for i in range(0,len(structure)-1):
        k=structure[i]*structure[i+1]
        x.append(k)
    x=sum(x)
    # x = shape of chromsome
    return 2*np.random.random((x,))-1 

#function to arrange the weights for each layers 
def weights_classifier(structure,chromosome):
    weights_list=[]
    for i in range(0,len(structure)-2):
        if i == 0:
            a=0
            b=structure[i]*structure[i+1]
        else:
            a=b
            b=a+(structure[i]*structure[i+1])
        r=structure[i]
        c=structure[i+1]
        #print('[',a,',',b,']','(',r,',',c,')')
        #print(len(chromosome))
        w=np.reshape(np.matrix(chromosome[a:b]),(r,c))
        
        weights_list.append(w)
    #print('weights: ',len(weights_list))   
    # weights_list = a list of aranged weights matrices  
    return weights_list

def sigmoid(x):
    return 1/(1+np.exp(-x))

def mse(train_output,answers):
    error=[]
    for i in range(len(answers)):
        mse=(train_output[i]-answers[i])**2
        error.append(mse)
    return error

def feed_forward(structure,chromosome,train_input):
    weights_list=weights_classifier(structure,chromosome)
    #for i in range(len(weights_list)):
        #print('weights list ',i,': ',weights_list[i].shape)
    for i in range(len(weights_list)):
        if i==0:
            output=sigmoid(np.dot(train_input,weights_list[i]))
        else:
            output=sigmoid(np.dot(output,weights_list[i]))
        #print('hl',[i],': ',weights_list[i])
    #print(output.shape)        
    return output
 
def learning(structure,chromosome,train_input):
    answers=[]
    for i in range(len(train_input)):
        output=feed_forward(structure,chromosome,train_input[i])
        answers.append(output)
    return np.array(answers)

In [294]:
population=[]

def populate(size):
    
    initial_pop=[None]*size
    
    for i in range(size):
        chromosome=chromosome_generetor(structure)
        initial_pop[i]=chromosome
    
    return initial_pop

def fitness(population,target,size,train_input,train_output,structure):
    
    score=[None]*size
    for index,chromosome in enumerate(population):
        
        answers=learning(structure,chromosome,train_input)
        error=mse(train_output,answers)
        score[index]=np.mean(error)-target,chromosome,index
        
    top_score=np.array(score)[np.array(score)[:,0].argsort()]
    #print(top_score)
    order=[None]*size
    for i in range(size):
        order[i]=top_score[i][1]
    
    top_snakes=np.array(order)[:int(size*0.1)]
    warrior=top_score[0]
    return top_snakes,warrior,answers


In [311]:
def mutation(child,m_rate):
    
    loop=int(round((len(child)/100)*m_rate))
    
    for i in range(loop):
        np.random.choice(child)+(2*np.random.random()-1)
    
    return child

def mate(population,size,m_rate):
    
    new_pop=[]
    for _ in range(int(size/2)):
        
        #print(len(population))
        p1=population[np.random.randint(0,len(population))]
        p2=population[np.random.randint(0,len(population))]
        

        child=[None]*len(p1)
        for i in range(0,len(child),2):
            if i == len(child)-1:
                child[i]=p1[i]
            else:
                child[i]=p1[i]
                child[i+1]=p2[i+1]
        child=mutation(child,m_rate)
        new_pop.append(child+(2*np.random.random((20,))-1))

        child=[None]*len(p1)
        for i in range(0,len(child),2):
            if i == len(child)-1:
                child[i]=p2[i]
            else:
                child[i]=p2[i]
                child[i+1]=p1[i+1]
        #child=mutation(child,m_rate)
        new_pop.append(child+(2*np.random.random((20,))-1))
            
    return new_pop


def cycle(generation,population,target,size,train_input,train_output,m_rate,structure):
    population=populate(size)
    #print(len(population))
    gen=0
    for _ in range(generation):
        gen+=1
        population,warrior,answers=fitness(population,target,size,train_input,train_output,structure)
        print('Generation: ',gen,'-----','Fittest: ',warrior[0])
        population=mate(population,size,m_rate)
        #print(population[0])

    return population
structure=[4,3,2,1,0]
size=1000
target=0
m_rate=100
population=cycle(80,population,target,size,train_input,train_output,m_rate,structure)

for i in range(len(answers)):
    print(np.round(answers[i],20),'===',train_output[i])

Generation:  1 ----- Fittest:  0.20703897251027617
Generation:  2 ----- Fittest:  0.19639132724826627
Generation:  3 ----- Fittest:  0.1871781058412511
Generation:  4 ----- Fittest:  0.17453703709000362
Generation:  5 ----- Fittest:  0.16353026964770903
Generation:  6 ----- Fittest:  0.14764568820678517
Generation:  7 ----- Fittest:  0.14000697015945196
Generation:  8 ----- Fittest:  0.13073280643349744
Generation:  9 ----- Fittest:  0.117578667274337
Generation:  10 ----- Fittest:  0.12126461584858111
Generation:  11 ----- Fittest:  0.10773385539956926
Generation:  12 ----- Fittest:  0.10556916742534153
Generation:  13 ----- Fittest:  0.09931360002914136
Generation:  14 ----- Fittest:  0.10131136876921742
Generation:  15 ----- Fittest:  0.09637100426139747
Generation:  16 ----- Fittest:  0.09360824600253267
Generation:  17 ----- Fittest:  0.09198781558151312
Generation:  18 ----- Fittest:  0.08813936932458352
Generation:  19 ----- Fittest:  0.08553766786496252
Generation:  20 ----- Fi

In [298]:
len(population[53])

20

In [96]:
structure=[4,3,2,1,0]
chromosome=chromosome_generetor(structure)

def learning(structure,chromosome,train_input):
    
    answers=[]
    for i in range(len(train_input)):
        output=feed_forward(structure,chromosome,train_input[i])
        answers.append(output)
    return np.array(answers)
answers=learning(structure,chromosome,train_input)


In [65]:
structure=[4,4,4,1,0]
chromosome=chromosome_generetor(structure)
for i in range(len(train_input)):
    output=feed_forward(structure,chromosome,train_input[i])
    print('output: ',output)

output:  [[0.41962001]]
output:  [[0.38160475]]
output:  [[0.3582655]]
output:  [[0.35785551]]
output:  [[0.32136635]]
output:  [[0.30074342]]
output:  [[0.31824609]]
output:  [[0.30316092]]
output:  [[0.26627595]]
output:  [[0.25149367]]
output:  [[0.26944252]]
output:  [[0.27677354]]
output:  [[0.33086415]]
output:  [[0.28281358]]
output:  [[0.2576655]]
output:  [[0.281456]]
